In [2]:
import nest_asyncio
nest_asyncio.apply()

In [3]:
import asyncio
import random

from agents import (
    Agent,
    function_tool,
)
from agents.extensions.handoff_prompt import prompt_with_handoff_instructions



@function_tool
def get_weather(city: str) -> str:
    """Get the weather for a given city."""
    print(f"[debug] get_weather called with city: {city}")
    choices = ["sunny", "cloudy", "rainy", "snowy"]
    return f"The weather in {city} is {random.choice(choices)}."


dutch_agent = Agent(
    name="Dutch",
    handoff_description="A dutch speaking agent.",
    instructions=prompt_with_handoff_instructions(
        "You're speaking to a human, so be polite and concise. Speak in Dutch.",
    ),
    model="gpt-4o-mini",
)

agent = Agent(
    name="Assistant",
    instructions=prompt_with_handoff_instructions(
        "You're speaking to a human, so be polite and concise. If the user speaks Dutch, handoff to the dutch agent.",
    ),
    model="gpt-4o-mini",
    handoffs=[dutch_agent],
    tools=[get_weather],
)

In [4]:
from agents.voice import SingleAgentVoiceWorkflow, VoicePipeline
pipeline = VoicePipeline(workflow=SingleAgentVoiceWorkflow(agent))

In [5]:
import numpy as np
import sounddevice as sd
from agents.voice import AudioInput

buffer = np.zeros(24000*3, dtype=np.int16)
audio_input = AudioInput(buffer=buffer)

result = await pipeline.run(audio_input)

player = sd.OutputStream(samplerate=24000, channels=1, dtype=np.int16)
player.start()

async for event in result.stream():
    if event.type == "voice_stream_event_audio":
        player.write(event.data)

In [ ]:
import asyncio
import random

from agents import Agent, function_tool
from agents.extensions.handoff_prompt import prompt_with_handoff_instructions
from agents.voice import (
    AudioInput,
    SingleAgentVoiceWorkflow,
    SingleAgentWorkflowCallbacks,
    VoicePipeline,
)

from util import AudioPlayer, record_audio

data = record_audio()